In [4]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from tqdm import tqdm
from IPython.display import clear_output
from itertools import product
import cv2 as cv

from utils import patient, dataset_INCan
from models import DBT_extractor

In [9]:
def transformation(im: sitk.Image, parameterMap_path:str, format='np', show=False):
    """apply transformation to an image

    Args:
        im (sitk.Image): image to be transformed
        parameterMap (sitk.ParameterMap): transformation settings
        format (str, optional): output format. Defaults to 'np'.

    Returns:
        np.array or sitk.Image: transformed image
    """
    transformixImageFilter = sitk.TransformixImageFilter()
    parameterMap = sitk.ReadParameterFile(str(parameterMap_path))
    transformixImageFilter.SetTransformParameterMap(parameterMap)
    transformixImageFilter.SetMovingImage(im)
    transformixImageFilter.Execute()
    im_result = transformixImageFilter.GetResultImage()
    clear_output(wait=show)
    if format=='sitk':
        return im_result
    elif format=='np':
        im_array = sitk.GetArrayFromImage(im_result)
        im_array = im_array.astype(np.uint16)
        return im_array
    else:
        raise ValueError('format must be "sitk" or "np"')    

def points_in_circle(radius):
    """iterable that yields all possible combinations of x and y coordinates within cicle of fixed radius

    Args:
        radius (int): radius of circle

    Yields:
        set: set with combinations
    """
    for x, y in product(range(int(radius) + 1), repeat=2):
        if x**2 + y**2 <= radius**2:
            yield from set(((x, y), (x, -y), (-x, y), (-x, -y),))

def translate_image(im:np.array, x_trans:int, y_trans:int):
    """translate image given x and y translation

    Args:
        im (np.array): image to be translated
        x_trans (int): x translation
        y_trans (int): y translation

    Returns:
        np.array: translated image
    """
    #get height and width
    height, width = im.shape[:2]
    #create translation matrix
    T = np.float32([[1, 0, x_trans], [0, 1, y_trans]])
    #translate
    img_translation = cv.warpAffine(im, T, (width, height))

    return img_translation

In [17]:
class budget_creator():
    def __init__(self):
        self._init_extractor()
        self.dataset = dataset_INCan()
        self._init_others()
        self.coord_list = list(self.points_in_circle(radius=2))
        self.rad_list = ['V', 'L', 'M']
        self.time_list = [1,2]

    def __repr__(self) -> str:
        return f'Budget creator using:\n{self.extractor}\n{self.dataset}\nAround points:{self.coord_list}'

    def _init_extractor(self):
        # define extractor
        config_file = str(repo_path / 'data/models/config_trained_R_101_30k.yaml')
        model_file = str(repo_path / 'data/models/model_final_R_101_omidb_30k_dbt9k_f12_gray.pth')
        min_score = 0.001 # minimum score to keep prediction
        self.extractor = DBT_extractor(config_file, model_file, min_score)

    def _init_others(self):
        saving_dir = repo_path / 'data/budget/substractions'
        saving_dir.mkdir(exist_ok=True, parents=True)
        im_saving_dir = repo_path / 'data/images/substractions'
        im_saving_dir.mkdir(exist_ok=True, parents=True)
    
    def points_in_circle(self, radius):
        """iterable that yields all possible combinations of x and y coordinates within cicle of fixed radius

        Args:
            radius (int): radius of circle

        Yields:
            set: set with combinations
        """
        for x, y in product(range(int(radius) + 1), repeat=2):
            if x**2 + y**2 <= radius**2:
                yield from set(((x, y), (x, -y), (-x, y), (-x, -y),))


In [19]:
budgeter = budget_creator()
budgeter.extractor

DBT_extractor(config_file=/home/ricardo/projects/Added_value_feature_uncertainty/data/models/config_trained_R_101_30k.yaml, model_file=/home/ricardo/projects/Added_value_feature_uncertainty/data/models/model_final_R_101_omidb_30k_dbt9k_f12_gray.pth, min_score=0.001)